# CFAR-10

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

batch_size = 4

trainset     = torchvision.datasets.CIFAR10(root='./data' ,train=True,download=True,transform=transform)

trainloader   = torch.utils.data.DataLoader(trainset , batch_size=batch_size,shuffle=True , num_workers=2)

testset     = torchvision.datasets.CIFAR10(root='./data' ,train=True,download=True , transform=transform)

testloader =torch.utils.data.DataLoader(testset , batch_size=batch_size ,shuffle=False ,num_workers=2)

classes = ('plane' , 'car' ,'bird' , 'cat' , 'deer' ,'dog' ,'frog' ,'horse' ,'ship' , 'truck')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img= img/2 +0.5
    npimg=img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
    
dataiter= iter(trainloader)
images , labels =dataiter.next()

imshow(torchvision.utils.make_grid(images))

print(''.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img= img/2 +0.5
    npimg=img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
    
dataiter= iter(trainloader)
images , labels =dataiter.next()

imshow(torchvision.utils.make_grid(images))

print(''.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

# 剛開始設定的初始值


In [ ]:
import torch
import torch.nn as nn
from math import ceil


base_model= [

    #expand_ratio , channels ,repeats , stride , kernel_size     
    [1, 16 ,1 ,1 ,3],
    [6, 24 ,2 ,2 ,3],
    [6, 40 ,2 ,2 ,5],
    [6, 80 ,3 ,2 ,3],
    [6, 112 ,3 ,1, 5],         
    [6, 192 ,4 ,2, 5],
    [6, 320 ,1 ,1 ,3],
             
]


phi_values = {
    
    "b0":(0,224,0.2),
    "b1":(0.5,240,0.2),
    "b2":(1,260,0.3),
    "b3":(2,300,0.3),
    "b4":(3,380,0.4),
    "b5":(4,456,0.4),
    "b6":(5,528,0.5),
    "b7":(6,600,0.5),

}

# CNNBlock

In [ ]:

import torch
import torch.nn as nn
from math import ceil






class CNNBlock(nn.Module):
  def __init__(
      self,in_channels, out_channels ,kernel_size,stride ,padding,groups=1 ):
    
    super(CNNBlock, self).__init__()
    self.cnn = nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        groups=groups,
    )
    self.bn = nn.BatchNorm2d(out_channels)
    self.silu= nn.SiLU() #SiLU <-> Swish

  def forward(self,x):
    return self.silu(self.bn(self.cnn(x)))






    

  

# SqueezeExcitation

In [ ]:
class SqueezeExcitation(nn.Module):
  def __init__(self,in_channels, reduced_dim):
    super(SqueezeExcitation, self).__init__()
    self.se =nn.Sequential(
     nn.AdaptiveAvgPool2d(1), 
     nn.Conv2d(in_channels, reduced_dim, 1),
     nn.SiLU(),
     nn.Conv2d(reduced_dim,in_channels, 1),
     nn.Sigmoid(),
    )   
  def forward(self, x):
     return x*self.se(x)



# InvertedResidualBlock

In [ ]:
class InvertedResidualBlock(nn.Module):
  def __init__(
      self,
      in_channels,
      out_channels,
      kernel_size,
      stride,
      padding,
      expand_ratio,
      reduction=4,
      survival_prob=0.8,

  ):
    super(InvertedResidualBlock, self).__init__()
    self.survival_prob = 0.8
    self.use_residual = in_channels == out_channels and stride == 1 
    hidden_dim  = in_channels * expand_ratio
    self.expand = in_channels != hidden_dim
    reduced_dim = int(in_channels / reduction)

    if self.expand:
      self.expand_conv = CNNBlock(in_channels,hidden_dim,kernel_size=3,stride=1,padding=1,
      )
    
    self.conv = nn.Sequential(
      CNNBlock(
          hidden_dim, hidden_dim, kernel_size,stride,padding, groups=hidden_dim,
      ),
      SqueezeExcitation(hidden_dim, reduced_dim),
      nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
      nn.BatchNorm2d(out_channels),
    )

  def stochastic_depth(self, x):
     if not self.training:
       return x
     binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.survival_prob
     return torch.div(x, self.survival_prob)*binary_tensor

    


  def forward(self, inputs):
     x= self.expand_conv(inputs) if self.expand else inputs 

     if self.use_residual:
       return self.stochastic_depth(self.conv(x)) + inputs
     else:
       return self.conv(x)

# Efficient Net

In [ ]:
class EfficientNet(nn.Module):
  def __init__(self, version, num_classes):
    super(EfficientNet,self).__init__()
    width_factor, depth_factor, dropout_rate=self.calculate_factors(version)
    last_channels = ceil(1280*width_factor)
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.features = self.create_features(width_factor, depth_factor, last_channels)
    self.classifier= nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(last_channels, num_classes),

    )


  def calculate_factors(self,version,alpha=1.2, beta=1.1):
    phi , res ,drop_rate = phi_values[version]
    depth_factor = alpha**phi
    width_factor = beta**phi
    return width_factor,depth_factor, drop_rate 

  def create_features(self,width_factor,depth_factor,last_channels):
    channels =int(32*width_factor)
    features =[CNNBlock(3,channels, 3, stride=2,padding=1)]
    in_channels = channels 

    for expand_ratio, channels, repeats , stride , kernel_size in base_model:
      out_channels = 4*ceil(int(channels*width_factor)/4)
      layers_repeats=  ceil(repeats * depth_factor)

      for layer in range(layers_repeats):
        features.append(
            InvertedResidualBlock(
                in_channels,
                out_channels,
                expand_ratio=expand_ratio,
                stride = stride if layer == 0 else 1,
                kernel_size=kernel_size,
                padding=kernel_size//2,
            )
            
        )
        in_channels = out_channels

      features.append(
        CNNBlock(in_channels, last_channels, kernel_size=1 , stride=1, padding=0)
      )

      return nn.Sequential(*features)

  def forward(self,x):
     x = self.pool(self.features(x))
     return self.classifier(x.view(x.shape[0], -1))
  
        



#  Evaulation &  comparision

In [ ]:

version ="b0"
num_examples, num_classes = 4, 10

net=EfficientNet(version=version,num_classes=num_classes)
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [ ]:


for epoch in range(100):
    
    running_loss= 0.0
    for i ,data in enumerate(trainloader, 0):
        
        inputs, labels =data
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        inputs, labels =inputs.to(device), labels.to(device)


        optimizer.zero_grad()
        outputs=net(inputs)
        loss =criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 ==1999:
            print(f'[{epoch+1},{i+1:5d}] loss: {running_loss / 2000 :.3f}')
            running_loss =0.0
            
print('Finished Training')

In [ ]:

PATH ='./clfar_net.pth'
torch.save(net.state_dict(),PATH)

In [ ]:
version ="b0"
num_examples, num_classes = 4, 10
net =EfficientNet(version=version,num_classes=num_classes)

net.load_state_dict(torch.load(PATH))

outputs = net(images)



dataiter =iter(testloader)
images , labels =dataiter.next()


imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ' ,''.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred =  {classname: 0 for classname in classes}

In [ ]:
with torch.no_grad():
  for data in testloader:
    images, labels = data
    outputs = net(images)
    _, predictions =torch.max(outputs, 1)

    for label, prediction in zip(labels, predictions):
      if label ==prediction:
        correct_pred[classes[label]] += 1
      total_pred[classes[label]] += 1



for classname, correct_count in correct_pred.items():
  accuracy =100*float(correct_count) / total_pred[classname]
  print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')